In [4]:
from tqdm.notebook import tqdm
import os
from glob import glob
import json
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple

import albumentations as A
import cv2
import numpy as np
import torch
from torch import nn 
from torchvision.ops import DeformConv2d 
from torch.utils.data import Dataset, DataLoader

import pycocotools.coco as coco
from pycocotools.cocoeval import COCOeval

In [ ]:
class Baseset(Dataset):

    def __init__(self, folders: List[str], h: int, w: int, transform=None):
        super().__init__()
        self.folders = folders  # e.g. ['data/F0']
        self.h = h  # image height
        self.w = w  # image width
        self.transform = transform

    def _float(self, x: float):
        return float(f'{x:0.2f}')


class Trainset(Baseset):

    def __init__(self, folders: List[str], h: int, w: int, transform=None):
        super().__init__(folders, h, w, transform)

    def __len__(self):
        return 0

    def __getitem__(self, idx):
        return None


class Testset(Baseset):

    def __init__(self, folders: List[str], h: int, w: int, transform=None):
        super().__init__(folders, h, w, transform)

    def __len__(self):
        return 0

    def __getitem__(self, idx):
        return None

In [27]:
def inspect_labels(folder_paths: str): 
    info = {}
    for folder_path in folder_paths:
        for lnr, label_fn in enumerate(glob(os.path.join(folder_path, 'Labels', '*.json'))):
            images = []  # labeled images
            for dict_ in json.load(open(label_fn, 'rb'))['Labels']:
                if isinstance(dict_, str):
                    continue
                img_path = os.path.join(folder_path, 'Images', str(lnr), dict_['imagefile'])
                if img_path not in images:
                    images.append(img_path)
            info[label_fn] = images
    return info  # collection of labeled image paths

# training folder names are from F0 to F11
info = inspect_labels([f'data/F{i}' for i in range(12)])

In [28]:
info

{'data/F0\\Labels\\Label1.json': ['data/F0\\Images\\0\\20191004_091820.tiff'],
 'data/F0\\Labels\\Label10.json': ['data/F0\\Images\\1\\20191004_092725.tiff'],
 'data/F0\\Labels\\Label11.json': ['data/F0\\Images\\2\\20191004_092727.tiff'],
 'data/F0\\Labels\\Label12.json': ['data/F0\\Images\\3\\20191004_092818.tiff'],
 'data/F0\\Labels\\Label13.json': ['data/F0\\Images\\4\\20191004_092920.tiff'],
 'data/F0\\Labels\\Label2.json': ['data/F0\\Images\\5\\20191004_091911.tiff'],
 'data/F0\\Labels\\Label3.json': ['data/F0\\Images\\6\\20191004_092019.tiff'],
 'data/F0\\Labels\\Label4.json': ['data/F0\\Images\\7\\20191004_092122.tiff'],
 'data/F0\\Labels\\Label5.json': ['data/F0\\Images\\8\\20191004_092226.tiff'],
 'data/F0\\Labels\\Label6.json': ['data/F0\\Images\\9\\20191004_092326.tiff'],
 'data/F0\\Labels\\Label7.json': ['data/F0\\Images\\10\\20191004_092421.tiff'],
 'data/F0\\Labels\\Label8.json': ['data/F0\\Images\\11\\20191004_092523.tiff'],
 'data/F0\\Labels\\Label9.json': ['data/F0\\Im